# Task 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import math
from datetime import datetime

In [ ]:
import pandas as pd

data_path = "/content/drive/MyDrive/ML/sample"

ap = pd.read_csv(data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag = pd.read_csv(data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei = pd.read_csv(data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf = pd.read_csv(data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd = pd.read_csv(data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [ ]:
# Display example observations
ap.head()

,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso
0,234,5697,M,2009.0,5,NaN,NaN,NaN,NaN,1949,2011.0,NaN
1,66,3539,M,2011.0,5,NaN,NaN,NaN,NaN,1933,2012.0,2015.0
2,100,3080,M,1988.0,5,NaN,NaN,NaN,NaN,1936,1991.0,NaN
3,112,3261,F,2014.0,5,NaN,NaN,NaN,NaN,1945,2014.0,NaN
4,160,9460,F,2014.0,5,NaN,NaN,NaN,NaN,1970,2016.0,NaN


In [ ]:
diag.head()

,idcentro,idana,data,codiceamd,valore
0,1,5,1980-01-01,AMD247,491.20
1,1,5,1986-01-01,AMD247,272.0
2,1,5,1987-01-01,AMD083,NaN
3,1,5,1987-01-01,AMD247,401
4,1,5,1997-12-01,AMD247,410


In [ ]:
elp.head()

,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.0
1,1,5,2005-01-18,AMD002,76.0
2,1,5,2005-01-18,AMD004,135.0
3,1,5,2005-01-18,AMD005,80.0
4,1,5,2005-06-06,AMD007,169.0


In [ ]:
ei.head()

,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [ ]:
pdf.head()

,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,82,782,2008-04-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG
1,82,782,2008-04-02,A10BA02,1.0,3,GLUCOPHAGE*60CPR RIV 1000MG
2,82,782,2008-04-02,A10BA02,1.0,5,GLUCOPHAGE*60CPR RIV 1000MG
3,82,782,2008-04-02,A10BB09,3.0,1,DIAMICRON*60CPR R.M. 30MG
4,82,782,2008-05-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG


In [ ]:
pdnf.head()

,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,34,2005-01-07,AMD090,S


In [ ]:
pnd.head()

,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


Borrar pacientes que:
  - fecha nacimiento > fecha primo accessso || fecha nacimiento > fecha muerte || fecha nacimiento > fecha diagnostico
  - fecha primo accesso > fecha muerte || fecha primo accesso > fecha diagnostico
  - fecha diagnostico > fecha muerte

Resto tablas, borrar pacientes que:
  - fecha < fecha nacimiento || fecha < fecha primo accesso || fecha > fecha muerte

En tablas de medicamentos:
  - fecha entre fecha diagnostico y fecha muerte

First we want to select from our tables the patients with dates that are valid (with the birth year smaller than the death year, the birth year smaller than the year of first access and so on)

In [ ]:
ap.shape

(250000, 12)

In [ ]:
ap_new = ap[(ap['annonascita'].notna()) & (ap['annoprimoaccesso'].notna()) & (ap['annodiagnosidiabete'].notna()) & (ap['annonascita']<=ap['annodecesso']) & (ap['annoprimoaccesso']<=ap['annodecesso']) & (ap['annodiagnosidiabete']<=ap['annodecesso'])]
print(ap_new.shape)

ap_new = pd.concat([ap_new, ap[ap['annodecesso'].isna()]])
print(ap_new.shape)

ap_new = ap_new[(ap_new['annonascita']<=ap_new['annodiagnosidiabete']) & (ap_new['annonascita']<=ap_new['annoprimoaccesso'])]
print(ap_new.shape)

(15872, 12)
(244824, 12)
(205229, 12)


In [ ]:
ap_test = ap_new.filter(items=['idcentro', 'idana'])
print(ap_test.shape)
ap_test.drop_duplicates()
print(ap_test.shape)

(205229, 2)
(205229, 2)


In [ ]:
#First we eliminate the patients that are no longer in the first table and the dates that do not exist
print(diag.shape)
diag = diag.merge(ap_test, on=['idcentro', 'idana'])
print(diag.shape)

diag = diag[diag['data'].notna()]
print(diag.shape)

#We assure that the dates are valid and that the year is first
diag['data'] = pd.to_datetime(diag['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
diag_new = diag.merge(ap_filter, on=['idcentro', 'idana'])
print(diag_new.shape)

#Now we take the rows of this table that have valid dates
diag_new = diag_new[diag_new['data'].dt.strftime("%Y").astype(float) <= diag_new['annodecesso']]
print(diag_new.shape)

diag_new = pd.concat([diag_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(diag_new.shape)

diag_new = diag_new[(diag_new['data'].dt.strftime("%Y").astype(float) >= diag_new['annoprimoaccesso']) & (diag_new['data'].dt.strftime("%Y").astype(float) >= diag_new['annonascita'])]
print(diag_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
diag_new = diag_new.filter(items=['idcentro', 'idana', 'data', 'codiceamd', 'valore'])
print(diag_new.shape)

(4978504, 5)
(4489796, 5)
(4489186, 5)
(4489186, 8)
(457599, 8)
(646956, 8)
(414088, 8)
(414088, 5)


In [ ]:
#First we eliminate the patients that are no longer in the first table
print(elp.shape)
elp = elp.merge(ap_test, on=['idcentro', 'idana'])
print(elp.shape)

elp = elp[elp['data'].notna()]
print(elp.shape)

#We assure that the dates are valid and that the year is first
elp['data'] = pd.to_datetime(elp['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
elp_new = elp.merge(ap_filter, on=['idcentro', 'idana'])
print(elp_new.shape)

#Now we take the rows of this table that have valid dates
elp_new = elp_new[elp_new['data'].dt.strftime("%Y").astype(float) <= elp_new['annodecesso']]
print(elp_new.shape)

elp_new = pd.concat([elp_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(elp_new.shape)

elp_new = elp_new[(elp_new['data'].dt.strftime("%Y").astype(float) >= elp_new['annoprimoaccesso']) & (elp_new['data'].dt.strftime("%Y").astype(float) >= elp_new['annonascita'])]
print(elp_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
elp_new = elp_new.filter(items=['idcentro', 'idana', 'data', 'codiceamd', 'valore'])
print(elp_new.shape)

(29398377, 5)
(25605464, 5)
(25605464, 5)
(25605464, 8)
(2009014, 8)
(2198371, 8)
(1889765, 8)
(1889765, 5)


In [ ]:
#First we eliminate the patients that are no longer in the first table
print(ei.shape)
ei = ei.merge(ap_test, on=['idcentro', 'idana'])
print(ei.shape)

ei = ei[ei['data'].notna()]
print(ei.shape)

#We assure that the dates are valid and that the year is first
ei['data'] = pd.to_datetime(ei['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
ei_new = ei.merge(ap_filter, on=['idcentro', 'idana'])
print(ei_new.shape)

#Now we take the rows of this table that have valid dates
ei_new = ei_new[ei_new['data'].dt.strftime("%Y").astype(float) <= ei_new['annodecesso']]
print(ei_new.shape)

ei_new = pd.concat([ei_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(ei_new.shape)

ei_new = ei_new[(ei_new['data'].dt.strftime("%Y").astype(float) >= ei_new['annoprimoaccesso']) & (ei_new['data'].dt.strftime("%Y").astype(float) >= ei_new['annonascita'])]
print(ei_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
ei_new = ei_new.filter(items=['idcentro', 'idana', 'data', 'codiceamd', 'valore'])
print(ei_new.shape)

(1062856, 5)
(973701, 5)
(973701, 5)
(973701, 8)
(66680, 8)
(256037, 8)
(63574, 8)
(63574, 5)


In [ ]:
#First we eliminate the patients that are no longer in the first table
print(pdf.shape)
pdf = pdf.merge(ap_test, on=['idcentro', 'idana'])
print(pdf.shape)

pdf = pdf[pdf['data'].notna()]
print(pdf.shape)

#We assure that the dates are valid and that the year is first
pdf['data'] = pd.to_datetime(pdf['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso', 'annodiagnosidiabete'])
pdf_new = pdf.merge(ap_filter, on=['idcentro', 'idana'])
print(pdf_new.shape)

#Now we take the rows of this table that have valid dates
pdf_new = pdf_new[pdf_new['data'].dt.strftime("%Y").astype(float) <= pdf_new['annodecesso']]
print(pdf_new.shape)

pdf_new = pd.concat([pdf_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(pdf_new.shape)

pdf_new = pdf_new[(pdf_new['data'].dt.strftime("%Y").astype(float) >= pdf_new['annoprimoaccesso']) & (pdf_new['data'].dt.strftime("%Y").astype(float) >= pdf_new['annonascita']) & (pdf_new['data'].dt.strftime("%Y").astype(float) >= pdf_new['annodiagnosidiabete'])]
print(pdf_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
pdf_new = pdf_new.filter(items=['idcentro', 'idana', 'data', 'codiceatc', 'quantita', 'idpasto', 'descrizionefarmaco'])
print(pdf_new.shape)

(7289926, 7)
(6248089, 7)
(6248089, 7)
(6248089, 11)
(544657, 11)
(734014, 11)
(513225, 11)
(513225, 7)


In [ ]:
#First we eliminate the patients that are no longer in the first table
print(pdnf.shape)
pdnf = pdnf.merge(ap_test, on=['idcentro', 'idana'])
print(pdnf.shape)

pdnf = pdnf[pdnf['data'].notna()]
print(pdnf.shape)

#We assure that the dates are valid and that the year is first
pdnf['data'] = pd.to_datetime(pdnf['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
pdnf_new = pdnf.merge(ap_filter, on=['idcentro', 'idana'])
print(pdnf_new.shape)

#Now we take the rows of this table that have valid dates
pdnf_new = pdnf_new[pdnf_new['data'].dt.strftime("%Y").astype(float) <= pdnf_new['annodecesso']]
print(pdnf_new.shape)

pdnf_new = pd.concat([pdnf_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(pdnf_new.shape)

pdnf_new = pdnf_new[(pdnf_new['data'].dt.strftime("%Y").astype(float) >= pdnf_new['annoprimoaccesso']) & (pdnf_new['data'].dt.strftime("%Y").astype(float) >= pdnf_new['annonascita'])]
print(pdnf_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
pdnf_new = pdnf_new.filter(items=['idcentro', 'idana', 'data', 'codiceamd', 'valore'])
print(pdnf_new.shape)

(569353, 5)
(509521, 5)
(509521, 5)
(509521, 8)
(48724, 8)
(238081, 8)
(47370, 8)
(47370, 5)


In [ ]:
#First we eliminate the patients that are no longer in the first table
print(pnd.shape)
pnd = pnd.merge(ap_test, on=['idcentro', 'idana'])
print(pnd.shape)

pnd = pnd[pnd['data'].notna()]
print(pnd.shape)

#We assure that the dates are valid with pd.to_datetime and that the year is first
pnd['data'] = pd.to_datetime(pnd['data'], yearfirst=True)

#Now we create a new table with the dates from the first and the second table
ap_filter = ap_new.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
pnd_new = pnd.merge(ap_filter, on=['idcentro', 'idana'])
print(pnd_new.shape)

#Now we take the rows of this table that have valid dates
pnd_new = pnd_new[pnd_new['data'].dt.strftime("%Y").astype(float) <= pnd_new['annodecesso']]
print(pnd_new.shape)

pnd_new = pd.concat([pnd_new, ap_filter[ap_filter['annodecesso'].isna()]])
print(pnd_new.shape)

pnd_new = pnd_new[(pnd_new['data'].dt.strftime("%Y").astype(float) >= pnd_new['annoprimoaccesso']) & (pnd_new['data'].dt.strftime("%Y").astype(float) >= pnd_new['annonascita'])]
print(pnd_new.shape)

#Now we take into account only the new diagnosis but we need to delete the extra columns
pnd_new = pnd_new.filter(items=['idcentro', 'idana', 'data', 'codiceamd', 'valore'])
print(pnd_new.shape)

(5573033, 5)
(4835132, 5)
(4835132, 5)
(4835132, 8)
(453570, 8)
(642927, 8)
(432217, 8)
(432217, 5)
